In [1]:
using LinearAlgebra
using TensorKit
using KrylovKit
using JSON
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_model.jl")
include("kagome_IPESS.jl")

D=8;
chi=10;
W=10;
tol=1e-6;

J1=1;
J2=0;
J3=0;
Jchi=0;
Jtrip=0;

parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);

A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

filenm="LS_D_"*string(D)*"_chi_40.json"
json_dict=read_json_state(filenm);

bond_tensor,triangle_tensor=construct_su2_PG_IPESS(json_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);

PEPS_tensor=bond_tensor;
@tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
A_unfused=PEPS_tensor;

U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
@tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];

init=Dict([("CTM", []), ("init_type", "PBC")]);
conv_check="singular_value";
@time CTM, AA_fused, U_L,U_D,U_R,U_U=CTMRG(A_fused,chi,conv_check,tol,init);

@time E_up, E_down=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, CTM, "E_triangle");
@time E_up_12, E_up_31, E_up_23, E_down_12, E_down_31, E_down_23=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, CTM, "E_bond");


"initialize CTM"

"start CTM iterations:"

 18.381457 seconds (39.06 M allocations: 2.448 GiB, 4.43% gc time, 94.68% compilation time)
CTMRG iteration: 1, CTMRG err: 2.0675324604252183
CTMRG iteration: 2, CTMRG err: 0.10308960444569149
CTMRG iteration: 3, CTMRG err: 0.01284009264784587
CTMRG iteration: 4, CTMRG err: 0.0029833510226271614
CTMRG iteration: 5, CTMRG err: 0.0007115976690582062
CTMRG iteration: 6, CTMRG err: 0.0007174015464860589
CTMRG iteration: 7, CTMRG err: 0.0005257568202744158
CTMRG iteration: 8, CTMRG err: 0.00033600709090235437
CTMRG iteration: 9, CTMRG err: 0.00019791003827619684
CTMRG iteration: 10, CTMRG err: 0.00011290484565708605
CTMRG iteration: 11, CTMRG err: 6.304507675332419e-5
CTMRG iteration: 12, CTMRG err: 3.4860088632304494e-5
CTMRG iteration: 13, CTMRG err: 1.9152477352803416e-5
CTMRG iteration: 14, CTMRG err: 1.0488630971295122e-5
CTMRG iteration: 15, CTMRG err: 5.732425153506735e-6
CTMRG iteration: 16, CTMRG err: 3.1295829862073303e-6
CTMRG iteration: 17, CTMRG err: 1.7074496746950487e-6
CTMRG

In [2]:
display((E_up+E_down)/3)
Tleft=CTM["Tset"][4];
Tright=CTM["Tset"][2];

@tensor O1[:]:=Tleft[-3,1,-1]*U_L[1,-2,-4];
@tensor O2[:]:=Tright[-1,1,-3]*U_R[-4,-2,1];


-0.4318744414103978 + 5.84501172087509e-18im

In [33]:
function HV_L_tensor(vl,A,mpo)
    if mpo==[]
        if numind(vl)==3
            @tensor vl[:]:=vl[-1,3,1]*A'[3,-2,2]*A[1,-3,2];
        elseif numind(vl)==2
            @tensor vl[:]:=vl[3,1]*A'[3,-2,2]*A[1,-3,2];
        end
    else
        if numind(vl)==5
            @tensor vl[:]:=vl[-1,7,5,3,1]*A'[7,-2,6]*mpo'[5,6,-3,4]*mpo[3,2,-4,4]*A[1,-5,2];
        elseif numind(vl)==4
            @tensor vl[:]:=vl[7,5,3,1]*A'[7,-1,6]*mpo'[5,6,-2,4]*mpo[3,2,-3,4]*A[1,-4,2];
        end
    end
    return vl
end

function HV_R_tensor(vr,A,mpo)
    if mpo==[]
        if numind(vr)==3
            @tensor vr[:]:=A'[-1,1,2]*A[-2,3,2]*vr[1,3,-3];
        elseif numind(vr)==2
            @tensor vr[:]:=A'[-1,1,2]*A[-2,3,2]*vr[1,3];
        end
    else
        if numind(vr)==5
            @tensor vr[:]:=A'[-1,7,6]*mpo'[-2,6,5,4]*mpo[-3,2,3,4]*A[-4,1,2]*vr[7,5,3,1,-5];
        elseif numind(vr)==4
            @tensor vr[:]:=A'[-1,7,6]*mpo'[-2,6,5,4]*mpo[-3,2,3,4]*A[-4,1,2]*vr[7,5,3,1];
        end
    end
    return vr
end

function left_eigenvector(A,mpo,type)

    if type=="A"
        HVfun1(x)=HV_L_tensor(x,A,[]);
        vl_init = permute(TensorMap(randn, space(A,1), space(A,1)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun1, vl_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    elseif type=="mpo_A"
        HVfun2(x)=HV_L_tensor(x,A,mpo);
        vl_init = permute(TensorMap(randn, space(A,1)*space(mpo,1),space(mpo,1)*space(A,1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun2, vl_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    end
end

function right_eigenvector(A,mpo,type)

    if type=="A"
        HVfun1(x)=HV_R_tensor(x,A,[]);
        vr_init = permute(TensorMap(randn, space(A,2), space(A,2)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun1, vr_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    elseif type=="mpo_A"
        HVfun2(x)=HV_R_tensor(x,A,mpo);
        vr_init = permute(TensorMap(randn, space(A,2)*space(mpo,3),space(mpo,3)*space(A,2)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun2, vr_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    end
end



function left_eigenvalue(A1,A2,n)
    function HV_L_A1A2(vl,A1,A2)
        if numind(vl)==3
            @tensor vl[:]:=vl[-1,3,1]*A1'[3,-2,2]*A2[1,-3,2];
        elseif numind(vl)==2
            @tensor vl[:]:=vl[3,1]*A1'[3,-2,2]*A2[1,-3,2];
        end
    return vl
    end
    HVA1A2fun(x)=HV_L_A1A2(x,A1,A2);
    vl_init = permute(TensorMap(randn, space(A1,1), space(A2,1)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
    eu,ev=eigsolve(HVA1A2fun, vl_init, 1,:LM,Arnoldi());
    @assert maximum(abs.(eu)) == abs(eu[1])
    eu=eu[1:n];
    return eu
end

left_eigenvalue (generic function with 1 method)

In [7]:
function impo_imps(mpo,A)
    unitary_mpo_A=unitary(fuse(space(mpo,1)⊗space(A,1)), space(mpo,1)⊗space(A,1));
    @tensor mpo_A[:]:=unitary_mpo_A[-1,2,1]*mpo[2,3,4,-3]*A[1,5,3]*unitary_mpo_A'[4,5,-2];
    return mpo_A
end

impo_imps (generic function with 1 method)

In [23]:
function itebd_ite(mpo,A,W,type)
    if type=="A"
        mpo_A=impo_imps(mpo,A);
        vl,_=left_eigenvector(mpo_A,[],"A");
        eu,ev=eig(permute(vl,(1,),(2,)));
        unitary_L=sqrt(eu)*ev';
        unitary_R=ev*pinv(sqrt(eu));
        @tensor AL[:]:=unitary_L[-1,1]*mpo_A[1,2,-3]*unitary_R[2,-2];
    elseif type=="mpo_A"
        vl,_=left_eigenvector(A,mpo,"mpo_A");
        eu,ev=eig(permute(vl,(2,1,),(3,4,)));
        unitary_L=sqrt(eu)*ev';
        unitary_R=ev*pinv(sqrt(eu));
        @tensor AL[:]:=unitary_L[-1,2,1]*mpo[2,3,4,-3]*A[1,5,3]*unitary_R[4,5,-2];
    end

    vr,_=right_eigenvector(AL,[],"A");

    U,S,_=tsvd(permute(vr,(1,),(2,)),trunc=truncdim(W));
    @tensor A_trun[:]:=U[1,-1]*AL[1,2,-3]*U'[-2,2];

    return A_trun
end


itebd_ite (generic function with 1 method)

In [37]:
function left_right_normalize(A)
    #left normalize
    vl,_=left_eigenvector(A,[],"A");
    eu,ev=eig(permute(vl,(1,),(2,)));
    unitary_L=sqrt(eu)*ev';
    unitary_R=ev*pinv(sqrt(eu));
    @tensor AL[:]:=unitary_L[-1,1]*A[1,2,-3]*unitary_R[2,-2];
    #transform right fixed point to be diagonal
    vr,eu=right_eigenvector(AL,[],"A");
    U,S,_=tsvd(permute(vr,(1,),(2,)));
    @tensor AL[:]:=U[1,-1]*AL[1,2,-3]*U'[-2,2];
    AL=AL/sqrt(eu);
    return AL
end


left_right_normalize (generic function with 1 method)

In [19]:

method="OO";
# ITEBD_boundary_groundstate(O1,O2,D,inv_tol):
    #use power method to obtain dominant boundary imps of nonhermitian MPO
    if method=="OO"
        @tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];
        U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));
        @tensor OO[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
    end
    #Initial state
    mps_virtual=SU₂Space(0=>1,1/2=>1,1=>1);mps_phy=space(OO,2);
    A=permute(TensorMap(randn, mps_virtual*mps_virtual', mps_phy),(1,2,3,),());
    AL=left_right_normalize(A);
    
    for cp=1:1
        AL_old=AL;
        @time if method=="O_O"
            A_trun_intermed=itebd_ite(O1,AL,W,"mpo_A");
            A_trun=itebd_ite(O2,A_trun_intermed,W,"mpo_A");
            AL=left_right_normalize(A_trun);
        elseif method=="OO"
            A_trun=itebd_ite(OO,AL,W,"mpo_A");#this "mpo_A" option is slightly faster when bond dimension is large
        end
        AL=left_right_normalize(A_trun);

        #dominant eigenvalue of transfer matrix
        E=left_eigenvalue(impo_imps(O2,impo_imps(O1,AL)),AL,1)
        println("E="*string(E))
        ov=abs(left_eigenvalue(AL,AL_old,1)[1])
        println("ov="*string(ov))
        if abs(ov-1)<1e-8
            break
        end
    #return AL
    end

  0.707679 seconds (2.21 M allocations: 211.692 MiB)
E=ComplexF64[3.0639519609227044 + 0.0im]
ov=3.0639519609226973


In [38]:
AL=left_right_normalize(AL);
display(abs(left_eigenvalue(AL,AL,1)[1]))

0.24999999999999994

1

2

3

4

5

6

7

8

9

10

In [134]:

# f(x)=HV_tensor(x,A);
# vl_init = permute(TensorMap(randn, SU₂Space(1/2=>1), space(A,1)'*space(A,1)), (1,2,3,),()); 
# eu,ev=eigsolve(f, vl_init, 1,:LM,Arnoldi());
# display(eu)


In [135]:
# A_dense=convert(Array,A);
# @tensor AA[:]:=conj(A_dense)[-1,-3,1]*A_dense[-2,-4,1];
# AA=reshape(AA,size(AA,1)*size(AA,2),size(AA,3)*size(AA,4));
# eigvals(AA)